In [1]:
import collections
import copy
import datetime
import itertools
import sys

import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
show_Ids = True
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay
Tournament = cibblbibbl.tournament.handler.default.Tournament
NormalPlayer = cibblbibbl.player.NormalPlayer
Season = cibblbibbl.season.Season
StarPlayer = cibblbibbl.player.StarPlayer

In [3]:
G.init()

In [5]:
Se = max(G.seasons).prev

In [12]:
ca = Se.continuous_activity()
for Te in sorted(ca, key=ca.get, reverse=True):
    print(f'{ca[Te]:>3} {Te}')

 24 [854600] Montetriste Noblemen
 24 [855120] Trantio Dragon Hawks
 24 [855753] Nejaz Djinn
 24 [854257] Grand Celestial Delegate
 24 [854507] Coven of the Mirrored Hall
 23 [859097] Bhubaneswar Rampage
 21 [867862] Everlasting Vultures of Settra
 21 [872475] True Roots of Khaine
 19 [874545] Stone Prince's Men
 19 [874302] Grasparossa B.B.C
 19 [874317] Misty Isle Gators
 19 [874452] Way of the Dragon
 19 [874966] Modryn's Midnight Meddlers
 18 [879131] Sartosa Black Flags
 18 [878945] Eternal Ecstasy
 17 [882382] Ali's Anarchists
 17 [886114] Eye and Arrow
 16 [891260] Library of Indra
 15 [896134] Rotfall Ironworks Rocketeers
 15 [896228] Gardeners of Nurgle
 15 [892248] Crimson Fins
 15 [895929] Fangsnapper's Clawpack
 14 [896564] Thoughts of Thoth
 14 [899682] K'r'shh Sl'ppr'ss
 14 [897353] Court of Shu
 14 [897379] Karak Kadrin Slayers
 14 [896972] Seekers of Transcendence
 13 [903654] Scions of Neferata
 12 [910344] Vlad's Vampheroes
 12 [906259] Livin' Thing
 12 [909848] Black

In [13]:
for Te in sorted(Se.gold_partner_teams(), key=ca.get, reverse=True):
    print(f'{ca[Te]:>3} {Te}')

 24 [854600] Montetriste Noblemen
 24 [855753] Nejaz Djinn
 24 [854507] Coven of the Mirrored Hall
 24 [855120] Trantio Dragon Hawks
 24 [854257] Grand Celestial Delegate
 23 [859097] Bhubaneswar Rampage
 21 [872475] True Roots of Khaine
 21 [867862] Everlasting Vultures of Settra
 19 [874317] Misty Isle Gators
 19 [874545] Stone Prince's Men
 19 [874452] Way of the Dragon
 19 [874966] Modryn's Midnight Meddlers
 19 [874302] Grasparossa B.B.C
 18 [878945] Eternal Ecstasy
 18 [879131] Sartosa Black Flags
 17 [886114] Eye and Arrow
 17 [882382] Ali's Anarchists
 16 [891260] Library of Indra


In [14]:
for Te in sorted(Se.silver_partner_teams(), key=ca.get, reverse=True):
    print(f'{ca[Te]:>3} {Te}')

 15 [896134] Rotfall Ironworks Rocketeers
 15 [895929] Fangsnapper's Clawpack
 15 [892248] Crimson Fins
 15 [896228] Gardeners of Nurgle
 14 [896564] Thoughts of Thoth
 14 [897353] Court of Shu
 14 [896972] Seekers of Transcendence
 14 [899682] K'r'shh Sl'ppr'ss
 14 [897379] Karak Kadrin Slayers
 13 [903654] Scions of Neferata
 12 [910344] Vlad's Vampheroes
 12 [906259] Livin' Thing
 12 [909848] Black Hearted Bloodseekers
 12 [906026] Karak Azog Outlaws
 12 [904377] Aurons
 12 [905924] Squill's Squig Pitt
 12 [907600] Deluge-Bearers
 12 [904411] Ynys Morion Mabsantwyr
 10 [864291] Norscan Curse
 10 [913702] Bordeleaux Chevaliers
 10 [914744] Mugrec's Mechanical Madmen
 10 [871787] Malignant Curators
  9 [923140] Dreadpeak's Delusionals
  9 [922672] Gazbag's Hunters
  9 [919662] Ursun Throat Singers
  9 [922608] Horsemen of The Great Kurgan
  9 [922869] Still Waters Run Deep
  8 [924441] Zharrduk Hellcannons
  8 [924841] Under-Nuln Bells
  8 [925779] Divine Sun
  8 [924405] Wolf Lands S

In [16]:
standings = Se.prestigestandings()
standings

[[1, 1205, (260, 210, 170, 45, 80, 95), Team(867862)],
 [2, 1000, (265, 140, 150, 45, 95, 65), Team(924441)],
 [3, 960, (160, 240, 140, 85, 80, 70), Team(874966)],
 [4, 955, (110, 230, 60, 135, 30, 70), Team(874452)],
 [5, 950, (240, 110, 200, 50, 65, 35), Team(874317)],
 [6, 950, (180, 180, 120, 115, 25, 80), Team(895929)],
 [7, 930, (180, 220, 220, 30, 30, 60), Team(879131)],
 [8, 905, (70, 150, 130, 110, 40, 85), Team(903654)],
 [9, 900, (265, 230, 70, 75, 75, 35), Team(924841)],
 [10, 855, (150, 160, 180, 95, 95, 45), Team(922608)],
 [11, 850, (160, 170, 150, 80, 65, 95), Team(874545)],
 [12, 820, (190, 210, 70, 65, 15, 85), Team(904411)],
 [13, 800, (200, 80, 60, 25, 55, 45), Team(859097)],
 [14, 800, (130, 170, 140, 50, 45, 60), Team(874302)],
 [15, 795, (70, 190, 170, 95, 20, 60), Team(896972)],
 [16, 790, (40, 210, 120, 65, 20, 75), Team(886114)],
 [17, 760, (180, 50, 120, 20, 30, 60), Team(854600)],
 [18, 745, (70, 140, 180, 110, 35, 80), Team(855120)],
 [19, 740, (120, 60, 15

In [6]:
standings = Se.prestigestandings()
standings

[[1, 1255, (260, 210, 170, 45, 80, 95), Team(867862)],
 [2, 1010, (160, 240, 140, 85, 80, 70), Team(874966)],
 [3, 1005, (110, 230, 60, 135, 30, 70), Team(874452)],
 [4, 1000, (240, 140, 150, 45, 95, 65), Team(924441)],
 [5, 1000, (240, 110, 200, 50, 65, 35), Team(874317)],
 [6, 980, (180, 220, 220, 30, 30, 60), Team(879131)],
 [7, 975, (180, 180, 120, 115, 25, 80), Team(895929)],
 [8, 930, (70, 150, 130, 110, 40, 85), Team(903654)],
 [9, 900, (240, 230, 70, 75, 75, 35), Team(924841)],
 [10, 900, (160, 170, 150, 80, 65, 95), Team(874545)],
 [11, 880, (150, 160, 180, 95, 95, 45), Team(922608)],
 [12, 850, (200, 80, 60, 25, 55, 45), Team(859097)],
 [13, 850, (130, 170, 140, 50, 45, 60), Team(874302)],
 [14, 845, (190, 210, 70, 65, 15, 85), Team(904411)],
 [15, 840, (40, 210, 120, 65, 20, 75), Team(886114)],
 [16, 820, (70, 190, 170, 95, 20, 60), Team(896972)],
 [17, 810, (180, 50, 120, 20, 30, 60), Team(854600)],
 [18, 795, (70, 140, 180, 110, 35, 80), Team(855120)],
 [19, 785, (40, 160,

In [8]:
ca = Se.continuous_activity()
for Te in sorted(ca, key=ca.get, reverse=True):
    v = ca[Te]
    if 16 <= v: continue
    if v <= 1: continue
    print(f'{ca[Te]:>2} {Te.name} ({Te.coach_name})')

15 Rotfall Ironworks Rocketeers (King_Mad)
15 Gardeners of Nurgle (EnglishJake)
15 Crimson Fins (Sweep12121)
15 Fangsnapper's Clawpack (flashman1234)
14 Thoughts of Thoth (Nohistor)
14 K'r'shh Sl'ppr'ss (det)
14 Court of Shu (SzieberthAdam)
14 Karak Kadrin Slayers (bigf)
14 Seekers of Transcendence (The_Carthaginian)
13 Scions of Neferata (Weresquid)
12 Vlad's Vampheroes (The_Demon)
12 Livin' Thing (GotrekSlayer)
12 Black Hearted Bloodseekers (Zartaz)
12 Aurons (awambawamb)
12 Squill's Squig Pitt (King_Joffrey)
12 Ynys Morion Mabsantwyr (ArthurWynne)
12 Karak Azog Outlaws (lautrehamon)
12 Deluge-Bearers (Cerebrosus)
10 Norscan Curse (darkjul)
10 Bordeleaux Chevaliers (Xelesar)
10 Mugrec's Mechanical Madmen (Pizzapocalypse)
10 Malignant Curators (ArrestedDevelopment)
 9 Dreadpeak's Delusionals (Lasgalen)
 9 Gazbag's Hunters (Christy)
 9 Ursun Throat Singers (LooseCable)
 9 Still Waters Run Deep (marksman0)
 9 Horsemen of The Great Kurgan (SpecialOne)
 8 Divine Sun (Throweck)
 8 Under-Nu